# Part 2: Creating a basic agent

Let's go through the fundamentals of creating an agent.
Here are the key concepts

These are
- **Function Definition** what tools the LLM has available to it.  
- **Function Call** a response from the model indicating a function call should be made.  
- **Function Response** the response we get from the function itself.  

And to tie this all together we need an **LLM Framework**

## How to do this
We're going to implement things one piece at time

## Our goal
Build the LLM framework from scratch so you can see every piece of how the system worls

**We're going to building the entire thing by hand**. 

Tjhis 

![functioncalling](img/function-calling-overview.png)

In [51]:
import logging
import re
import json

## Our first LLM tool
This is an (arbitrary) tool that we want our LLM to use. While this one is simple the tool can be anything. We'll start with this one because its 

In [52]:
def get_weather(city):
    logging.info("Called weather function %s", city)
    if city.lower() in {"Austin", "Sydney"}:
        return "sunny"
    else:
        return "cloudy"

In [53]:
get_weather("Austin")

'cloudy'

## AI Chat Bot
Let's now start buildlding

In [62]:
from ollama import chat
from ollama import ChatResponse

# 12b is better so try and use it first
model = 'gemma3:12b'
# model = 'gemma3:4b'


# Note, the argument model_prompt is specific here
def model_call(model_prompt):
    
    response: ChatResponse = chat(model=model, messages=[
      {
        'role': 'user',
        'content': model_prompt,
      },
    ])
    return response['message']['content']

user_prompt = "Say hello to the class"

# Note, the argument user_prompt is specific here
model_call(user_prompt)

"Hello everyone! It's great to be here with you all today! 😊 \n\n\n\nHow's everyone doing?"

## Add a system prompt
System prompts 

In [63]:
def augmented_model_call(system_prompt, user_prompt, print_prompt = False):
    combined_prompt = f"{system_prompt}\n{user_prompt}"

    if print_prompt:
        print(combined_prompt)
    
    return model_call(combined_prompt)

In [64]:
# This is injected by the LLM application behind the scenes

system_prompt = '''Talk like a pirate to everyone. \n '''
augmented_model_call(system_prompt, user_prompt, print_prompt=True)

Talk like a pirate to everyone. 
 
Say hello to the class


"Ahoy, mateys! Shiver me timbers, gather 'round, ye scurvy dogs!\n\nAvast there, class! Welcome aboard! May yer studies be fruitful and yer knowledge as vast as the seven seas! Arrr!"

## Tool Call System Prompt

In [68]:
system_prompt = '''
You have the following functions available
 def get_weather(city: str)
   """Given a city returns the weather for that city""

 If you call this function return the json [{"city": city}] and nothing else
 otherwise respond normally
'''

In [69]:
user_prompt = "What's the weather in Sydney?"
augmented_model_call(system_prompt, user_prompt)

'```json\n[{"city": "Sydney"}]\n```\n'

In [70]:
user_prompt = "How's the Austin forecast looking today?"
augmented_model_call(system_prompt, user_prompt)

'```json\n[{"city": "Austin"}]\n```\n'

In [71]:
user_prompt = "How's the London weather looking today?"
augmented_model_call(system_prompt, user_prompt)

'```json\n[{"city": "London"}]\n```\n'

## Parsing the Response
Now that we can see our model is aware of the tools we need to build a framework around it to process results.

We need a basic condition
1. If there is no function call return the response to the user
2. If there is a function call then we need to
    a. Call the tool to get new information
    b. Either return the response back to the user directly, or reinject it back to hte model

In [75]:
pattern = f"```json\n(.*?)\n``"

def parse_response(model_response):
    if tool_call := re.search(pattern, model_response):
        return json.loads(tool_call.groups(0)[0])[0]

model_response = '```json\n[{"city": "Austin"}]\n```\n'
parse_response(model_response)

{'city': 'Austin'}

## Put all the pieces together: Full tool Interaction 

In [79]:
def chat_interaction(user_prompt):

    system_prompt = '''
    You have the following functions available
     def get_weather(city: str)
       """Given a city returns the weather for that city""
    
     If you call this function return the json [{"city": city}] and nothing else
     otherwise respond normally
    '''

    # Get a model response. Right now we don't know if its a function call or chat response
    model_response = augmented_model_call(system_prompt, user_prompt)

    # Regex to see if we have the json which indicates a function call
    function_call_json = parse_response(model_response)

    # If it's not a function call return the response
    if not function_call_json:
        return model_response
    
    # Since we detect a function call
    weather = get_weather(function_call_json["city"])

    # We have a choice here
    # We could return the weather directly to the user
    # But for a nicer experience let's reinject in the LLM to the response
    function_response_prompt = f"The weather in {function_call_json['city']} is {weather}, tell me the weather nicely"

    # We already checked for weather so we don't need to go again
    model_response = model_call(function_response_prompt)
    return model_response

chat_interaction("Can you say hi to class?")

'Hi class! \n\n(And if I were to call `get_weather("London")`, I would return `[{"city": "London"}]`.)'

In [82]:
chat_interaction("What's the weather today in London?")

"The sky in London is looking a bit grey and cloudy today! It's a cozy kind of day, perfect for a warm drink and a good book. 😊"

In [81]:
chat_interaction("How are things looking in Austin?")

"The sky in Austin is looking a little cloudy today! It's a cozy, soft kind of day. 😊"

## 🎯 Recap: What We Learned  

### Function Calls are nothing magical
* We rely on the models "reasoning" to understand when to make a function call or not

### The model doesn't call a function itself
* It's the framework that handles the function call

## Your turn: Do this with a real api

In [2]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [3]:
LONDON_LATITUDE = 51.5074 # Approximate latitude for London
LONDON_LONGITUDE = -0.1278 # Approximate longitude for London (West is negative)
get_weather(LONDON_LATITUDE, LONDON_LONGITUDE)

10.8

## Tool Call versus Agents

![functioncalling](img/AlwaysHasBeen.jpg)
Source: https://huggingface.co/blog/tiny-agents

Agents have a lot of definitions, the simplest however is that they're a model that can call tools in a for loop.